<a href="https://colab.research.google.com/github/ydsbald/KNN_books_recomandation/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Charger les données
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convertir les labels en valeurs binaires
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Séparer les caractéristiques et les labels
X_train = train_df['message']
y_train = train_df['label']
X_test = test_df['message']
y_test = test_df['label']

# Tokenizer les données textuelles
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")  # Augmenter le vocabulaire
tokenizer.fit_on_texts(X_train)

# Convertir les textes en séquences et les padder
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')  # Augmenter la longueur maximale
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')

# Créer le modèle amélioré
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=100),  # Embedding plus grand
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),  # LSTM bidirectionnelle
    keras.layers.GlobalMaxPooling1D(),  # Réduire la dimensionnalité
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),  # Dropout ajusté
    keras.layers.Dense(1, activation='sigmoid')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Entraîner le modèle
history = model.fit(
    X_train_padded, y_train,
    epochs=20,  # Augmenter le nombre d'époques
    validation_data=(X_test_padded, y_test),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]  # Ajouter des callbacks
)

# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Function to predict messages based on model
def predict_message(pred_text):
    # Tokenize and pad the input text
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = pad_sequences(sequence, maxlen=50, padding='post', truncating='post')

    # Get the model's prediction
    prediction = model.predict(padded_sequence)
    probability = prediction[0][0]  # Extract the probability of the message being spam

    # Determine the label based on the probability
    label = "spam" if probability > 0.5 else "ham"

    # Return the probability and label as a list
    return [probability, label]

# Test the function
pred_text = "how are you doing today"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
